# ODL and Tomography

This notebook demonstrates how to use ODL for tomographic problems.

## Contents

### Geometry Concepts

* Continuous geometries
* Coordinate systems
* Functions to query geometry properties

### Construction of Geometries

* Convenience functions for quick prototyping
* Class constructors for fine-grained control

### Code Examples

* Something

## Parametrization of Geometries

A more detailed exposure of handling of geometries in ODL can be found in [the online documentation](https://odlgroup.github.io/odl/guide/geometry_guide.html).


ODL adopts a continuous view on acuisition geometries (this should not be too surprising anymore at this point). In principle, acquisition geometries define manifolds on which data is defined, but ODL makes no attempt at encoding those manifolds. Instead, **parameters** that would be mapped to the data manifold by a *parametrization* are used to represent a geometry.

---

**Example:**
The 2D Ray Transform can be defined as

$$
    \begin{align*}
        & R: L^2(\mathbb{R}^2) \to L^2(M),\quad Rf(\theta, v) = \int_{\mathbb{R}} f(t\theta + v)\, \mathrm{d}t, \\
        & |\theta| = 1,\ \langle \theta, v \rangle = 0.
    \end{align*}
$$

In this case, the data manifold would be the set

$$
    M = \{(\theta, v)\, |\, |\theta| = 1,\ \langle \theta, v \rangle = 0\}.
$$

However, it is much more intuitive and convenient (and allows a simpler implementation) to use *angles* and *shifts on detector* to encode the same information:

$$
    \begin{align*}
        & R: L^2(\mathbb{R}^2) \to L^2([0, \pi] \times \mathbb{R}),\quad Rf(\varphi, s) = \int_{\mathbb{R}} f\bigl( t\theta(\varphi) + s\theta^\perp(\varphi) \bigr)\, \mathrm{d}t, \\
        & \theta(\varphi) = (-\sin\varphi, \cos\varphi),\ \theta^\perp(\varphi) = (\cos\varphi, \sin\varphi).
    \end{align*}
$$

<img src="parallel2d_geom.png" alt="Parallel 2D Geometry" style="width: 600px;"/>

---

Some generic principles can be extracted from the example that allow to represent other, potentially more complex geometries:

* *A continuous function maps parameters to points on the data manifold.*

  In the example, this function is $(\varphi, s) \mapsto \bigl( \theta(\varphi), s \theta^\perp(\varphi) \bigr)$.

* *Parameters come from Cartesian products of intervals.*

  In the example, these intervals are $[0, \pi]$ and $\mathbb{R}$, although in practice, one would use a finite interval for the detector, of course.

* *Parameters can be separated into a "motion" part and a "detector" part.*

  This split is very natural for applications where the object is stationary and the detector moves (as in medical imaging). Other applications require a bit of phantasy to fit them to this acquisition model.

## Construction of Geometries

The `Geometry` classes in ODL give a lot of control over the definition of the vectors that determine their key properties. For cases when this flexibility is not needed, there exist convenience functions to quickly generate various common geometries.

### Convenience Factories for Geometries

The following functions exist for the creation of geometries (all in the `odl.tomo` subpackage):

* `parallel_beam_geometry`
* `cone_beam_geometry`
* `helical_geometry`

All helpers require as input a `space`, which should be a `DiscreteLp` instance. Its primary purpose is to determine the dimension of the geometry to create, but it is also used to find the required size of the detector to fully cover the reconstruction domain, and to find the minimum number of angles and detector pixels such that the Nyquist sampling condition is satisfied ([NW2001], pages 72--74).

Both the number of angles and the shape of the detector can be overridden via additional arguments.

Let's look at an example. We define the reconstruction domain as $[-1, 1] \times [-1, 1]$ and use a $200 \times 200$ grid for its discretization.

In [5]:
import odl

X = odl.uniform_discr([-1, -1], [1, 1], shape=(200, 200))

Next we create a parallel beam geometry suitable for this space. Here, the detector must be at least as large as the diameter of the reconstruction domain, to ensure that all rays intersecting the domain fall onto the detector. We can check that this is the case:

In [14]:
geom = odl.tomo.parallel_beam_geometry(X)
print('Geometry type:', type(geom))
print('Number of angles:', geom.motion_partition.size)
print('Number of detector elements:', geom.det_partition.size)
print('Detector extent:', float(geom.det_partition.extent))

Geometry type: <class 'odl.tomo.geometry.parallel.Parallel2dGeometry'>
Number of angles: 445
Number of detector elements: 285
Detector extent: 2.8284271247461903


Indeed, the length of the detector is $2\sqrt{2}$, enough to fully cover the given domain.

The conditions on the step sizes $\Delta\varphi$ and $\Delta s$ for angles and detector for full sampling are as follows [NW2001]:

$$
    \Delta\varphi \leq \frac{\pi}{\Omega}, \quad \Delta s \leq \frac{\pi}{\rho\Omega},
$$

where $\rho$ is the radius of the function support ($\sqrt{2}$ here), and $\Omega$ is the (essential) bandlimit of the discrete reconstruction space. We can choose $\Omega$ using the pixel size $\Delta x$ of the space:

$$
    \Omega = \frac{\pi}{\Delta x}.
$$

Thus, in our case, we expect that the generated geometry has an angle step of $\Delta\varphi \approx \Delta x / \sqrt{2}$ and a detector step size of $\Delta s \approx \Delta x$. Let's check this:

In [13]:
print('dx / dphi:', X.cell_sides[0] / geom.motion_partition.cell_sides[0])  # should be sqrt(2)
print('dx / ds:', X.cell_sides[0] / geom.det_partition.cell_sides[0])  # should be 1

dx / dphi: 1.41647899352
dx / ds: 1.00762716319


For small problems, these default choices are reasonable, but for larger problems, the number of angles can become unrealistically large, and it makes sense to manually set the number of angles.
Likewise, when the number of detector pixels is given by the application, one can set it manually. This will lead to larger pixels (full coverage is always guaranteed):

In [15]:
geom = odl.tomo.parallel_beam_geometry(X, num_angles=120)
print('Number of angles:', geom.motion_partition.size)
print('dx / dphi:', X.cell_sides[0] / geom.motion_partition.cell_sides[0])

Number of angles: 120
dx / dphi: 0.381971863421


In [16]:
geom = odl.tomo.parallel_beam_geometry(X, num_angles=120, det_shape=128)
print('Number of detector elements:', geom.det_partition.size)
print('Detector extent:', float(geom.det_partition.extent))  # should still be 2*sqrt(2)
print('dx / ds:', X.cell_sides[0] / geom.det_partition.cell_sides[0])

Number of detector elements: 128
Detector extent: 2.8284271247461903
dx / ds: 0.452548339959


#### `motion_partition`? `det_partition`?

In the examples above, discretization properties of the geometry were retrieved through the `motion_partition` and `det_partition` properties of the geometry. Those kinds of objects will be covered in the next section since the class constructors of geometries expect them as arguments.

### Geometry Class Constructors

The convenience factories for geometries are useful for creating quick "throw-away" geometry objects, but only cover a limited set of possibilities. For instance, it is not possible to create geometries with limited angle, data truncation or non-standard orientations. For such more advanced purposes, the class constructors should be used.

#### Partitions

The minimum information needed to construct a geometry is the set from which parameters can be taken, together with their discretizations (plus additional configuration depending on the type of geometry). In ODL, these "discretizations of intervals" or of Cartesian products of intervals are called **partitions**.

For a 1D interval, a partition is a division into small subintervals, together with a set of grid points. Most of the time, the grid points will be the midpoints of the subintervals, but not always.

---

**Example:**

The role of partitions is best explained with numerical integration. Suppose we have an interval $[c_0, c_n]$ subdivided into intervals $[c_i, c_{i+1}],\ i = 0, \dots, n-1$. For each subinterval, we can apply the [midpoint rule](https://en.wikipedia.org/wiki/Midpoint_rule) to integrate a function $f$:

$$
    \int_{c_i}^{c_{i+1}} f(t)\, \mathrm{d}t \approx (\underbrace{c_{i+1} - c_i}_{=:\Delta c_i})\, f\bigg( \underbrace{\frac{c_i + c_{i+1}}{2}}_{=:g_i} \bigg)
$$

We can sum up the values to get an approximation to the integral over the whole interval:

$$
    \int_{c_0}^{c_n} f(t)\, \mathrm{d}t \approx \sum_{i=0}^{n-1} \Delta c_i\, f(g_i).
$$

Thus the grid points $g_i$ play the role of the *sampling points* of the function $f$, whereas the lengths of the cells, $\Delta c_i$, act as *summation weights*.

---

In ODL, the combination of grid points and subintervals (or rather the multi-dimensional version of it) are called *partitions*. They are used extensively in `DiscreteLp` spaces and in `Geometry` objects.

Let's look at some examples of how to create and query properties of partitions. The single most used function to create them is `uniform_partition`:

In [22]:
part = odl.uniform_partition(0, 1, 10)
print('Partitioned interval:', part.set)
print('Subinterval boundaries:', part.cell_boundary_vecs[0])
print('Grid points:', part.grid.coord_vectors[0])

Partitioned interval: [0.0, 1.0]
Subinterval boundaries: [ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ]
Grid points: [ 0.05  0.15  0.25  0.35  0.45  0.55  0.65  0.75  0.85  0.95]


### Coordinate Systems

To make handling of different coordinate systems consistent and easier to understand, ODL geometries all define a **default configuration** and rules for achieving non-standard configurations. This means that all vectors that determine a configuration of a geometry have a default value and transition to other settings through translation and rotation.

For instance, in the above example, the choice of the vectors $\theta(\varphi) = (-\sin\varphi, \cos\varphi)$ and $\theta^\perp(\varphi) = (\cos\varphi, \sin\varphi)$ was somewhat arbitrary. Any other pair of orthogonal (unit) vectors would have been equally valid.

For the 2D parallel beam geometry, the above choice is the default. At angle $\varphi = 0$, rays are parallel to the Y axis ($\theta(0) = (0, 1)$), the detector coordinate increases along the +X axis ($\theta^\perp(0) = (1, 0)$), and for increasing $\varphi$, this right-hand system performs a counter-clockwise rotation.

Similar rules apply to other geometries in 2D and 3D.

## References

[NW2001] Natterer, F and Wuebbeling, F. Mathematical Methods in Image Reconstruction. SIAM, 2001. https://dx.doi.org/10.1137/1.9780898718324